In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
np.random.seed(1)

In [2]:
import os
os.getcwd()

'/Users/dileepunnikrishnan'

# Importing and Cleaning table

In [3]:
tnn = pd.read_csv('/Users/dileepunnikrishnan/desktop/NN_source_file(1).csv', index_col=[0])
tnn.shape

(1647, 34)

In [4]:
tnn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1647 entries, 1 to 1647
Data columns (total 34 columns):
Name                  1647 non-null object
CPMRN                 1629 non-null object
Month.of.Admission    1647 non-null object
Age                   1647 non-null int64
Gender                1647 non-null object
Hospital              1647 non-null object
History               1647 non-null object
Surgery               1647 non-null object
Vent.mode             1623 non-null object
GCS                   1647 non-null int64
Temparature           1647 non-null object
HR                    1647 non-null float64
SpO2                  1647 non-null int64
SBP                   1647 non-null float64
MAP                   1647 non-null float64
RR                    1647 non-null int64
FiO2                  1647 non-null float64
PaO2                  1647 non-null float64
PaCO2                 1647 non-null float64
pH                    1647 non-null float64
A.a.gradient          1647 non

In [5]:
tnn['Temparature'] = pd.to_numeric(tnn['Temparature'], errors='coerce')
tnn.Temparature = tnn.Temparature.replace(np.nan, 98.6)
sum(tnn.Temparature.isnull())

0

In [6]:
tnn['Month.of.Admission'].unique()

array(['July 2018', 'August 2018', 'September 2018', 'October 2018',
       'November 2018', 'December 2018', 'January 2019', 'February 2019',
       'July 0201', 'January 2018', 'August 2015', 'February 2018',
       'February 0219'], dtype=object)

In [7]:
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('July 0201', 'July 2018')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('January 2018', 'January 2019')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('August 2015', 'August 2018')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('February 2018', 'February 2019')
tnn['Month.of.Admission'] = tnn['Month.of.Admission'].replace('February 0219', 'February 2019')

In [8]:
tnn['Vent.mode'].value_counts()

RA           555
Mask         407
Cannula      332
ETT          171
High Flow     95
Trach         63
Name: Vent.mode, dtype: int64

In [9]:
tnn['Vent.mode'] = tnn['Vent.mode'].replace(np.nan, 'RA')

In [10]:
tnn2 = tnn.drop(['Name', 'CPMRN', 'History', 'SBP'], axis=1)

In [11]:
tnn_trim = tnn.drop(['Name', 'CPMRN', 'History', 'SBP', 'Month.of.Admission', 'Gender', 'Hospital'], axis=1)

In [12]:
tnn2.columns

Index(['Month.of.Admission', 'Age', 'Gender', 'Hospital', 'Surgery',
       'Vent.mode', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2',
       'PaO2', 'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets',
       'K', 'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate',
       'INR', 'Survival'],
      dtype='object')

In [13]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
def scaleColumns(df, cols_to_scale):
    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])))
    return df
tnn_scaled = scaleColumns(tnn2, ['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2', 
                                          'PaO2', 'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets', 
                                         'K', 'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate', 
                                         'INR'])
tnt_scaled = scaleColumns(tnn_trim, ['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2', 
                                          'PaO2', 'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets', 
                                         'K', 'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate', 
                                         'INR'])

/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/dileepunnikrishnan/anaconda3/envs

### Changing dtypes of numerics from float64 to float32

In [14]:
names = np.array(tnn_scaled.columns)
for i in range(len(names)):
    if tnn_scaled[names[i]].dtype == 'float64':
        tnn_scaled[names[i]] = tnn_scaled[names[i]].astype('float32')
names2 = np.array(tnt_scaled.columns)
for i in range(len(names2)):
    if tnt_scaled[names2[i]].dtype == 'float64':
        tnt_scaled[names2[i]] = tnt_scaled[names2[i]].astype('float32')
tnn_scaled.info()
tnt_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1647 entries, 1 to 1647
Data columns (total 30 columns):
Month.of.Admission    1647 non-null object
Age                   1646 non-null float32
Gender                1647 non-null object
Hospital              1647 non-null object
Surgery               1647 non-null object
Vent.mode             1647 non-null object
GCS                   1646 non-null float32
Temparature           1646 non-null float32
HR                    1646 non-null float32
SpO2                  1646 non-null float32
MAP                   1646 non-null float32
RR                    1646 non-null float32
FiO2                  1646 non-null float32
PaO2                  1646 non-null float32
PaCO2                 1646 non-null float32
pH                    1646 non-null float32
A.a.gradient          1646 non-null float32
HCO3                  1646 non-null float32
Hb                    1646 non-null float32
TLC                   1646 non-null float32
Platelets         

In [15]:
tnn3 = pd.get_dummies(tnn_scaled, drop_first=True)
tnt3 = pd.get_dummies(tnt_scaled, drop_first=True)

In [16]:
tnn3 = tnn3.dropna()
tnt3 = tnt3.dropna()

In [17]:
print(tnn3.columns)
print(tnt3.columns)

Index(['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO2', 'PaO2',
       'PaCO2', 'pH', 'A.a.gradient', 'HCO3', 'Hb', 'TLC', 'Platelets', 'K',
       'Na', 'Serum.Cr', 'Blood.Urea', 'Bili', 'Urine.output', 'Lactate',
       'INR', 'Month.of.Admission_December 2018',
       'Month.of.Admission_February 2019', 'Month.of.Admission_January 2019',
       'Month.of.Admission_July 2018', 'Month.of.Admission_November 2018',
       'Month.of.Admission_October 2018', 'Month.of.Admission_September 2018',
       'Gender_Male', 'Hospital_Brains-MVJ', 'Hospital_Brains-SSNMC',
       'Hospital_HCG DR', 'Hospital_HCG EKO', 'Hospital_HCG Hubli',
       'Hospital_HCG Vijayawada', 'Hospital_MRNH', 'Hospital_Prashant',
       'Surgery_Emergency surgery', 'Surgery_Non-Operative', 'Vent.mode_ETT',
       'Vent.mode_High Flow', 'Vent.mode_Mask', 'Vent.mode_RA',
       'Vent.mode_Trach', 'Survival_Expired'],
      dtype='object')
Index(['Age', 'GCS', 'Temparature', 'HR', 'SpO2', 'MAP', 'RR', 'FiO

In [18]:
print(tnn3.shape)
print(tnt3.shape)

(1646, 48)
(1646, 32)


In [94]:
X = tnn3.iloc[:, 0:47].values; Y = tnn3.iloc[:, 47].values.reshape(-1, 1); 
print(X.shape); print(Y.shape);

(1646, 47)
(1646, 1)


In [20]:
x = tnt3.iloc[:, 0:31].values; y = tnt3.iloc[:, 31].values.reshape(-1, 1);
print(x.shape); print(y.shape);

(1646, 31)
(1646, 1)


In [233]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, stratify=Y)

In [234]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, stratify=y)

## Keras Model
## Performance in full data

In [271]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras import regularizers
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [329]:
model = Sequential()
model.add(Dense(300, activation='relu', input_shape=(47,)))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(50, kernel_regularizer=regularizers.l2(0.1), activation='relu'))
model.add(Dense(2, activation='softmax'))

In [330]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [331]:
model.fit(X_train, Y_train, batch_size=100, epochs=20)

Epoch 1/20
823/823 [==============================] - 12s 14ms/step - loss: 16.7609 - acc: 0.8870
Epoch 2/20
823/823 [==============================] - 1s 762us/step - loss: 13.6424 - acc: 0.9514
Epoch 3/20
823/823 [==============================] - 0s 582us/step - loss: 10.8341 - acc: 0.9514
Epoch 4/20
823/823 [==============================] - 1s 611us/step - loss: 8.5427 - acc: 0.9514
Epoch 5/20
823/823 [==============================] - 0s 589us/step - loss: 6.6615 - acc: 0.9514
Epoch 6/20
823/823 [==============================] - 1s 726us/step - loss: 5.1844 - acc: 0.9514
Epoch 7/20
823/823 [==============================] - 1s 789us/step - loss: 4.0372 - acc: 0.9514
Epoch 8/20
823/823 [==============================] - 1s 664us/step - loss: 3.1284 - acc: 0.9514
Epoch 9/20
823/823 [==============================] - 1s 626us/step - loss: 2.4171 - acc: 0.9514
Epoch 10/20
823/823 [==============================] - 1s 746us/step - loss: 1.8853 - acc: 0.9514
Epoch 11/20
823/823 [=====

In [332]:
Y_tr_pred_prob = model.predict(X_train)
Y_ts_pred_prob = model.predict(X_test)

In [333]:
y_tr_pred_prob = Y_tr_pred_prob[:,1]
y_ts_pred_prob = Y_ts_pred_prob[:,1]
print(roc_auc_score(Y_train, y_tr_pred_prob))
print(roc_auc_score(Y_test, y_ts_pred_prob))

0.9994891443167305
0.5561224489795918


## Performance in trim data

In [302]:
model = Sequential()
model.add(Dense(300, activation='relu', input_shape=(31,)))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dense(300, kernel_regularizer=regularizers.l2(0.01), activation= 'relu'))
model.add(Dense(50, kernel_regularizer=regularizers.l2(0.1), activation='relu'))
model.add(Dense(2, activation='softmax'))

In [303]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [304]:
model.fit(x_train, y_train, batch_size=100, epochs=20)

Epoch 1/20
823/823 [==============================] - 10s 12ms/step - loss: 13.8308 - acc: 0.9526
Epoch 2/20
823/823 [==============================] - 0s 438us/step - loss: 11.0058 - acc: 0.9526
Epoch 3/20
823/823 [==============================] - 0s 454us/step - loss: 8.6276 - acc: 0.9526
Epoch 4/20
823/823 [==============================] - 0s 476us/step - loss: 6.6941 - acc: 0.9526
Epoch 5/20
823/823 [==============================] - 1s 728us/step - loss: 5.1646 - acc: 0.9526
Epoch 6/20
823/823 [==============================] - 1s 677us/step - loss: 3.9643 - acc: 0.9526
Epoch 7/20
823/823 [==============================] - 0s 466us/step - loss: 3.0353 - acc: 0.9526
Epoch 8/20
823/823 [==============================] - 0s 581us/step - loss: 2.3245 - acc: 0.9526
Epoch 9/20
823/823 [==============================] - 1s 767us/step - loss: 1.7797 - acc: 0.9526
Epoch 10/20
823/823 [==============================] - 0s 515us/step - loss: 1.3660 - acc: 0.9526
Epoch 11/20
823/823 [======

In [305]:
Y_tr_pp = model.predict(x_train)
Y_ts_pp = model.predict(x_test)
y_tr_pred_prob = Y_tr_pp[:,1]
y_ts_pred_prob = Y_ts_pp[:,1]
print(roc_auc_score(y_train, y_tr_pred_prob))
print(roc_auc_score(y_test, y_ts_pred_prob))

0.9774659863945578
0.6819604086845465


## Hyperparameter Tuning

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [25]:
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}

In [26]:
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)

In [27]:
logreg_cv.fit(x_train, y_train)

/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/dileepunnikrishnan/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.00000e-05, 8.48343e-05, 7.19686e-04, 6.10540e-03, 5.17947e-02,
       4.39397e-01, 3.72759e+00, 3.16228e+01, 2.68270e+02, 2.27585e+03,
       1.93070e+04, 1.63789e+05, 1.38950e+06, 1.17877e+07, 1.00000e+08])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [29]:
print("Tuned LR parameters: {}".format(logreg_cv.best_params_))
print("Best score is: {}".format(logreg_cv.best_score_))

Tuned LR parameters: {'C': 1e-05}
Best score is: 0.9513888888888888
